# Analise da base do DataSUS sobre casos de dengue no primeiro semestre de 2025
Buscamos a base com dados de dengue do portal [https://opendatasus.saude.gov.br/gl/dataset/arboviroses-dengue](https://opendatasus.saude.gov.br/gl/dataset/arboviroses-dengue/resource/5c9132a9-77c2-4b15-8afc-a43c58fc9ec0?inner_span=True)

O Dataset contém dados de casos de dengue coletados na primeira metade de 2025.

Com base nessas informações queremos responder as seguintes perguntas:

1. Se existe relação entre o municipio em que uma pessoa mora com a probabilidade de contrair dengue?
2. Quais sintomas estão mais relacionados a dengue? 

Ao final, avaliaremos qual modelo é melhor para ajudar médicos na detecção de possíveis casos de Dengue e seu tipo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.expand_frame_repr', False) 

df = pd.read_csv("dados_dengue.csv")
df.head()

# Análise das caracteristicas da base

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
for coluna in df.columns:
    tipo = df[coluna].dtypes
    print(f'{coluna}: {tipo}')

Como a base é bastante extensa e com muitas colunas selecionamos as colunas que aparentemente estão mais relacionadas a nossa hipótese.
Neste caso selecionamos a coluna **CLASSI_FIN** como target, pois ela se a classificação final do caso.

Além dela selecionamos as colunas:

- 'FEBRE' 
- 'MIALGIA' 
- 'CEFALEIA'
- 'EXANTEMA'
- 'VOMITO'
- 'NAUSEA'
- 'DOR_COSTAS'
- 'CONJUNTVIT'
- 'ARTRITE'
- 'ARTRALGIA'
- 'PETEQUIA_N'
- 'PETEQUIA_N'
- 'LACO' 
- 'DOR_RETRO'


In [ ]:
sintomas = ['FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LACO', 'DOR_RETRO']


Essas colunas indicam quais sistomas o paciente apresentava no momento da investigação, seus valores são:
1. Sim 
2. Não

Por fim selecionamos a coluna **ID_MUNICIP** que se refere ao Cód. do município onde está localizada a unidade de saúde (ou outra fonte notificadora) que realizou a notificação. O nome está associado ao código na tabela de municípios.



In [ ]:
df = df[['CLASSI_FIN', 'ID_MUNICIP', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LACO', 'DOR_RETRO']]
df.head()

In [ ]:
df['CLASSI_FIN'].value_counts()

Os valores dessa coluna representam as seguintes informações:

- 10. Dengue
- 11. Dengue com sinais de alarme
- 12. Dengue grave
- 8. Descartado

# Analise dos Dados

In [ ]:
colunas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
plt.figure(figsize=(20, 15))

for i, coluna in enumerate(colunas_numericas, 1):
    plt.subplot(4, 5, i)  # Ajuste o layout conforme o número de colunas
    sns.histplot(df[coluna], kde=True, bins=30, color='orange') # type: ignore
    plt.title(f'Distribuição de {coluna}')
    plt.xlabel(df[coluna].name)
    plt.ylabel('Frequência')

plt.tight_layout()
plt.show()

Mapemos a correlação das variaveis e o mapa de calor

In [ ]:
plt.figure(figsize=(14, 12)) #ajusta o tamanho 

sns.heatmap(df.corr(), 
            cmap='coolwarm',      # Define o esquema de cores (azul para correlações negativas, vermelho para positivas)
            annot=True,           # Adicionar valores numéricos
            fmt=".2f",            # Formato com 2 casas decimais
            linewidths=0.5,       # Adicionar linhas entre células
            cbar_kws={'shrink': 0.8}  # Ajustar barra de cores
            )

plt.xticks(rotation=45, ha='right')  # Coloca os nomes do campos do eixo x 45 graus
plt.yticks(rotation=0)  # Coloca os nomes do campos do eixo y na horizontal
plt.tight_layout()      # Ajustar layout automaticamente, evitar que título ou legendas sejam cortados nas bordas
plt.title('Matriz de Correlação entre Variáveis', fontsize=14)
plt.show()

Usando a correlação comum as variaves não parecem estar correlacionadas individualmente com o nosso target.
Pesquisando descobrimos que quando suas variáveis são categóricas (tipo “febre”, “dor”, “positivo”), a correlação tradicional (como .corr() do pandas) não funciona bem — porque ela mede relação linear entre variáveis numéricas continuas. Mas existem outras formas de avaliar a associação entre duas variáveis categóricas.

Nesse caso escolhemos o **Qui-quadrado de independência** que é uma ferramenta estatística usada para verificar se existe alguma associação entre duas variáveis categóricas. Ele responde à pergunta: "Será que essas variáveis estão relacionadas ou são independentes?"

# Testes Qui-quadrado (independência)

Se o p-valor for < 0.05, indica associação estatística significativa.

In [ ]:
from scipy.stats import chi2_contingency

alpha = 0.05
for coluna in df.columns:
    tabela = pd.crosstab(df[coluna], df['CLASSI_FIN'])
    chi2, p, dof, expected = chi2_contingency(tabela)
    # Compare the p-value to the significance level
    print(f"p-valor `{coluna}`: {p}")

Pelo teste de Qui-quadrado parece que 'ID_MUNICIP', 'FEBRE', 'MIALGIA', 'CEFALEIA' e 'VOMITO' estão relacionadas com a classificação final da doença no dataframe onde ons nulos foram removidos.
Ja no dataframe preenchido com a moda 'ID_MUNICIP', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'ARTRITE' e 'DOR_RETRO' são as colunas relacionadas.

# Normalizar dos dados


Primeiro verificamos se existem valores nulos na base.

In [ ]:
df.isnull().sum()

Nesse caso decidimos remover os nulo da nossa variavel target, pois é um dado categórico e queremos que ele condiza com a realiadade. Usar técnicas para preenche-lo poderia causar variações no resultado da predição.

In [ ]:
df = df.dropna()
print(df.isnull().sum())
print(df.shape)

Segundo vamos transformar todas as colunas do dataframe em tipo inteiro para melhor representar os dados.

In [ ]:
for coluna in df.columns:
    df[coluna] = df[coluna].fillna(0).astype(int)
df.head()

Como os sintomas são na verdade valores booleanos realizamos a normalização dos valores '2' para '0' para que eles fiquem em escala binária.

In [ ]:
for sintoma in sintomas:
    df.loc[df[sintoma] == 2, sintoma] = 0
df.head()

Como os valores 10, 11, 12 da coluna **CLASSI_FIN** representam que o paciente tem dengue e o valor 8 que não tem. Vamos colocar essa coluna em escala binário também.

In [ ]:
df.loc[df['CLASSI_FIN'] == 10, 'CLASSI_FIN'] = 1
df.loc[df['CLASSI_FIN'] == 11, 'CLASSI_FIN'] = 1
df.loc[df['CLASSI_FIN'] == 12, 'CLASSI_FIN'] = 1
df.loc[df['CLASSI_FIN'] == 8, 'CLASSI_FIN'] = 0
df.head(10)

# Treino e Avaliação dos Modelos

Separação base de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split


x = df[['ID_MUNICIP', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'VOMITO']]
y = df['CLASSI_FIN']
     

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,
                                                    random_state=42)
     

print("Total base de treino: ", len(x_train))
print("Total base de teste: ", len(y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                            roc_curve, auc, precision_recall_curve) 

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

resultados = {}
for nome, modelo in models.items():
    print(f"\nTreinando modelo: {nome}")

    modelo.fit(x_train, y_train)

    y_pred = modelo.predict(x_test)
    
    # Calculando métricas
    acuracia = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Armazenando resultados
    resultados[nome] = {
        'acuracia': acuracia,
        'report': report,
        'modelo': modelo,
        'predicoes': y_pred
    }
    
    print(f"Acurácia: {acuracia:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

In [ ]:
import pickle

with open('model.pkl', 'wb') as arquivo:
    pickle.dump(resultados["KNN"]["modelo"], arquivo)

A acuracia está boa, mas a precisão não. Vamos aplicar oversampling para os casos de não dengue.

In [ ]:


from imblearn.over_sampling import SMOTE
     

# Aplicar SMOTE para oversampling da classe minoritária
oversample = SMOTE()
x_train_os, y_train_os = oversample.fit_resample(x_train, y_train)
     
print("Total base de treino: ", len(x_train))
print("Total base de teste: ", len(y_test))

print("Total base de treino oversampling: ", len(x_train_os))
print("Total base de teste oversampling: ", len(y_train_os))

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                            roc_curve, auc, precision_recall_curve) 

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

resultados = {}
for nome, modelo in models.items():
    print(f"\nTreinando modelo: {nome}")

    modelo.fit(x_train_os, y_train_os)

    y_pred = modelo.predict(x_test)
    
    # Calculando métricas
    acuracia = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Armazenando resultados
    resultados[nome] = {
        'acuracia': acuracia,
        'report': report,
        'modelo': modelo,
        'predicoes': y_pred
    }
    
    print(f"Acurácia: {acuracia:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

KNN foi o algoritmo que se comportou melhor. Mas vamos testar os hiperparametros para otimiza-lo.

In [ ]:
error = [] #armazenar os erros

# Calculating error for K values between 1 and 15
for i in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train_os, y_train_os)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))

In [ ]:


plt.figure(figsize=(12, 6))
plt.plot(range(1, 15), error, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('Error Rate K Value')
plt.xlabel('K Value')
plt.ylabel('Mean Error')



In [ ]:
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(x_train_os, y_train_os)
y_pred = knn.predict(x_test)
acuracia = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)

In [ ]:
print(f"Acurácia: {acuracia:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))